In [ ]:
"""Workstation for clean_hdf5.py"""

# pylint: disable=redefined-outer-name, expression-not-assigned, import-error, not-callable, pointless-statement, no-value-for-parameter, undefined-variable, unused-argument, missing-docstring, line-too-long

In [ ]:
from __future__ import annotations

import shutil
from pathlib import Path

import h5py

In [ ]:
from epiclass.utils.clean_hdf5 import load_bed, preprocess_bed, print_h5py_structure

In [ ]:
print_h5py_structure(
    str(
        Path.home()
        / "Projects/epilap/input/hdf5/hg38_2022-epiatlas/100kb_all_none/8a4eb67b1e30d37eede02a28f691a917_100kb_all_none_value.hdf5"
    )
)

In [ ]:
home = Path("/home/local/USHERBROOKE/rabj2301/Projects")
input_dir = home / "epilap/input"
hdf5_list_path = input_dir / "hdf5_list" / "100kb_all_none_10samples.list"

In [ ]:
og_hdf5_path = Path(
    "/home/local/USHERBROOKE/rabj2301/Projects/epilap/input/hdf5/hg38_2022-epiatlas/100kb_all_none/67959517fd1cd857151bbabca3abc74f_100kb_all_none_value.hdf5"
)
hdf5_path = og_hdf5_path.parent / (og_hdf5_path.stem + "_0blklst.hdf5")
shutil.copy(og_hdf5_path, hdf5_path)

bin_resolution = 100 * 1000  # Replace with the appropriate bin resolution value
blacklist_path = "/home/local/USHERBROOKE/rabj2301/Projects/epilap/input/filter/hg38_unified_blacklist.bed"
blacklist_bed = load_bed(blacklist_path)
blacklist_chromosome_intervals = preprocess_bed(blacklist_bed)


with h5py.File(hdf5_path, "r+") as file:
    header = list(file.keys())[0]
    hdf5_data = file[header]
    for chromosome, dataset in hdf5_data.items():  # type: ignore
        for i, value in enumerate(dataset):
            position = i * bin_resolution
            if is_position_in_blacklist(
                position,
                position + bin_resolution,
                blacklist_chromosome_intervals,
                chromosome,
                verbose=True,
            ):
                # print(i)
                dataset[i] = 0

    file.close()